In [1]:
import os

from langchain.agents import Tool
from langchain.memory import ChatMessageHistory, ConversationBufferWindowMemory
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import AgentExecutor
from langchain import hub
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.tools.render import render_text_description

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#(https://makersuite.google.com/)
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv('GENERATIVE_LANGUAGE_API_KEY')
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    convert_system_message_to_human=True,
    handle_parsing_errors=True,
    temperature=0.6,
    max_tokens= 1000,
    safety_settings = {
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    },
)

# Caso queira utilizar uma LLM local com o Ollama
# from langchain_ollama.llms import OllamaLLM
# llm = OllamaLLM(model="llama3.2", temperature=0.6, max_tokens=200)

In [4]:
from planing_tools import weatherapi_forecast_periods, query_rag
ddg_search = DuckDuckGoSearchAPIWrapper()

planing_tools = [
    Tool(
        name="DuckDuckGo Search",
        func=ddg_search.run,
        description="""Essa ferramenta DEVE ser utilizada para buscar eventos relevantes no período fornecido pelo usuário. 
        Ela é útil para obter informações sobre eventos ou atividades especiais que estão acontecendo na cidade de destino nas datas que o usuário informou. 
        O modelo deve usá-la para complementar as sugestões de atividades."""
    ),
    Tool(
        name="Weather Forecast",
        func=weatherapi_forecast_periods,
        description="""Essa ferramenta deve ser usada apenas quando o modelo já tiver todas as informações necessárias para gerar o roteiro e o usuário já tiver fornecido as datas exatas. 
        Se o usuário pedir informações para um período de tempo, o modelo deve consultar o clima separadamente para cada dia desse período. 
        Ela deve ser usada quando a previsão do tempo for um fator importante para o planejamento, mas apenas depois que o modelo tiver claro os detalhes do itinerário."""
    ),
    Tool(
        name="Query RAG",
        func=query_rag,
        description="""Esta ferramenta deve ser usada quando o modelo souber a cidade de destino e os interesses do usuário, com o objetivo de fornecer informações sobre pontos turísticos e atrações que se alinham com esses interesses. 
        O modelo deve utilizar essa ferramenta para sugerir atividades e lugares específicos a visitar, baseados na cidade e nos interesses fornecidos."""
    )
]

In [5]:
planing_agent_prompt = hub.pull("tales/agente_turismo")

In [6]:
print(planing_agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'data_atual', 'input', 'tool_names', 'tools'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'tales', 'lc_hub_repo': 'agente_turismo', 'lc_hub_commit_hash': '0ea92afeadc4b6828f8d39bf902b1a032ed5effb56827eeace35e81d36ce8852'} template='Seu nome é Sofia. Você é uma assistente pessoal de inteligência artificial de classe mundial, especializada em viagens e turismo. Você é altamente avançada e inteligente, muito útil e eficiente em fornecer informações, planejar roteiros personalizados e resolver problemas relacionados a viagens. Sofia é um modelo AELM (Modelo de Linguagem de Execução Automática) com foco em planejamento de viagens.\n\n"Sofia" foi projetada para auxiliar usuários em todas as etapas de suas aventuras, desde a escolha do destino até a criação de itinerários detalhados e dicas práticas. Como modelo de linguagem, "Sofia" é capaz de gerar texto semelhante ao humano com base nas informações fornecidas pelos usuá

In [7]:
planing_prompt = planing_agent_prompt.partial(
    tools=render_text_description(planing_tools),
    tool_names=", ".join([t.name for t in planing_tools]),
)
llm_with_stop = llm.bind(stop=["\nObservation"])

In [8]:
history = ChatMessageHistory()
memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")

C:\Users\Tales\AppData\Local\Temp\ipykernel_18232\275711510.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=20, chat_memory=history, memory_key="chat_history")


In [9]:
from datetime import datetime
hoje = datetime.today()

data_formatada = hoje.strftime("%Y/%m/%d")
# Pega o dia da semana em inglês
dia_da_semana_ingles = hoje.strftime("%A")

# Dicionário para mapear os dias da semana de inglês para português
dias_da_semana_pt = {
    "Monday": "segunda-feira",
    "Tuesday": "terça-feira",
    "Wednesday": "quarta-feira",
    "Thursday": "quinta-feira",
    "Friday": "sexta-feira",
    "Saturday": "sábado",
    "Sunday": "domingo"
}

# Traduz o dia da semana para português
dia_da_semana_pt = dias_da_semana_pt.get(dia_da_semana_ingles, dia_da_semana_ingles)

# Formata a data para utilizar no prompt
data_atual = f"{dia_da_semana_pt}, {data_formatada}"

In [10]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_log_to_str(x["intermediate_steps"]),
        "chat_history": lambda x: x["chat_history"],
        "data_atual": lambda x: data_atual,
    }
    | planing_prompt
    | llm_with_stop
    | ReActSingleInputOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=planing_tools, verbose=True, memory=memory)

In [11]:
print(data_atual)

domingo, 2024/12/29


In [12]:
response = agent_executor.invoke({"input": f"""
                                  Boa noite
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Boa noite! Para começarmos a planejar sua viagem, qual o seu destino?
```


> Finished chain.
Boa noite! Para começarmos a planejar sua viagem, qual o seu destino?
```


In [13]:
response = agent_executor.invoke({"input": f"""
                                  Eu estou planejando visitar Natal no Rio Grande do Norte
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Natal é uma excelente escolha. Para começarmos a planejar sua viagem, quantos dias você pretende passar por lá?
```


> Finished chain.
Ótimo! Natal é uma excelente escolha. Para começarmos a planejar sua viagem, quantos dias você pretende passar por lá?
```


In [14]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo passar da quarta dia 01 de janeiro até o domingo
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Perfeito! De 01 de janeiro a 06 de janeiro, então.  Para te ajudar a criar o roteiro ideal, quais atividades ou experiências você está buscando em Natal? (Por exemplo: praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?)
```


> Finished chain.
Perfeito! De 01 de janeiro a 06 de janeiro, então.  Para te ajudar a criar o roteiro ideal, quais atividades ou experiências você está buscando em Natal? (Por exemplo: praias, restaurantes, eventos, museus, história, cultura, vida noturna, natureza, ou algo mais?)
```


In [15]:
response = agent_executor.invoke({"input": f"""
                                  Eu planejo visitar praias e restaurantes regionais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores restaurantes regionais e praias em Natal, Rio Grande do NortePRAIA DA REDINHA
A última praia ao norte de Natal é uma antiga vila de pescadores, cujo destaque é o 
Mercado Público. A tradição ali é comer a ginga com tapioca (peixe frito que vem no 
palito, dentro de uma tapioca). A Igreja de Nossa Senhora dos Navegantes, construída 
com pedras dos arrecifes, e o Aquário Natal são outros pontos de interesse.
PRAIA DO MEIO
Situada entre a Praia do Forte 
e a Praia dos Artistas, sedia 
anualmente, durante o Réveillon, 
a tradicional festa de Iemanjá. Era 
considerada a praia mais turística 
da cidade até os anos 1980, 
devido às formações de piscinas 
naturais da região.
PRAIA DOS ARTISTAS
A 5 km do centro de Natal, é uma  
das mais agitadas da cidade. Suas 
ondas altas são ótimas para surfe 
e outros esportes náuticos. Com

---

PRAIA A PRAIA
A orla urbana de Natal é marcada por completa estrutura 

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Praias e restaurantes regionais são ótimas escolhas para sua viagem a Natal. Para criar um roteiro ainda melhor, você tem um orçamento em mente para a viagem?
```


> Finished chain.
Ótimo! Praias e restaurantes regionais são ótimas escolhas para sua viagem a Natal. Para criar um roteiro ainda melhor, você tem um orçamento em mente para a viagem?
```


In [16]:
response = agent_executor.invoke({"input": f"""
                                  Sem incluir Hóteis pretendo gastar no máximo 600 reais
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores restaurantes regionais em Natal, Rio Grande do Norte, com preços acessíveisNA HORA 
DA FOME...
Espere saborear frutos 
do mar de todos os 
jeitos e também pratos 
típicos nordestinos nos 
restaurantes de Natal
É uma referência de 
gasto médio por pessoa, 
sem incluir as bebidas.
FAIXA DE PREÇOS
$ – de R$ 40 a R$ 60
$$ – de R$ 61 a R$ 90
$$$ – de R$ 91 a R$ 150
TÁBUA DE CARNE
Prepara deliciosa comida regional na 
tábua, que serve três pessoas, além de 
sobremesas típicas. Também tem uma 
unidade na Via Costeira. Cliente CVC ganha 
15% de desconto nos pratos. Av. Eng. 
Roberto Freire, 2.396, Ponta Negra. $
Ô BAR 
Serve comida variada 
em um ambiente perfeito 
para famílias, que 
tem até brinquedoteca. Alguns 
destaques são as moquecas e a 
carne de sol na chapa com queijo, 
além das porções típicas de bar. 
Av. Praia de Ponta Negra, 9184, 
Ponta Negra. $$
MULLE

---

COMPRAS
As riquezas naturais d

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo! Com um orçamento de R$600,00 para 4 dias em Natal, focados em praias e restaurantes regionais, posso criar um roteiro incrível para você.  Para te dar sugestões mais precisas, você teria alguma preferência por alguma região específica de Natal (ex: Ponta Negra, Pipa)?  E qual o seu estilo de viagem? Prefere algo mais relaxante ou agitado?
```


> Finished chain.
Ótimo! Com um orçamento de R$600,00 para 4 dias em Natal, focados em praias e restaurantes regionais, posso criar um roteiro incrível para você.  Para te dar sugestões mais precisas, você teria alguma preferência por alguma região específica de Natal (ex: Ponta Negra, Pipa)?  E qual o seu estilo de viagem? Prefere algo mais relaxante ou agitado?
```


In [17]:
response = agent_executor.invoke({"input": f"""
                                  Eu não possuo nenhuma alergia
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo!  Sem alergias, isso facilita as sugestões de restaurantes.  Para finalizar, você prefere uma viagem mais relaxante ou agitada?  Isso irá influenciar nas sugestões de atividades.
```


> Finished chain.
Ótimo!  Sem alergias, isso facilita as sugestões de restaurantes.  Para finalizar, você prefere uma viagem mais relaxante ou agitada?  Isso irá influenciar nas sugestões de atividades.
```


In [18]:
response = agent_executor.invoke({"input": f"""
                                  Agitada.
                                  """})

print(response["output"])



> Entering new AgentExecutor chain...


c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? Yes
Action: Query RAG
Action Input: melhores praias e restaurantes regionais em Natal, Rio Grande do Norte, para uma viagem agitada de 4 dias.8 DIAS (7 NOITES) 
6º Dia Você descobre por que o Rio 
Grande do Norte é a terra do camarão na 
Barra do Cunhaú, grande produtora do 
crustáceo e local de praias desertas.
7º Dia Feche sua viagem na Baía Formosa, 
onde você pode fazer um passeio opcional 
de bugue até a divisa com a Paraíba, 
parando na Lagoa da Coca-Cola.
8º Dia Hora de voltar para casa.
DICAS DE ROTEIROS
Os passeios destacados na sugestão abaixo estão nas páginas 10 a 13
4 DIAS (3 NOITES)
1º Dia: Chegada à cidade. Almoce no 
Restaurante Farofa d’Água, em Ponta 
Negra, para se encantar logo de cara 
com a vista do Morro do Careca.
2º Dia: Faça o Passeio Panorâmico 
para descobrir o melhor de Natal 
e curtir a Praia de Porto Mirim, 
onde você pode fazer um passeio 
opcional de bugue, com brincadeiras

---

são para charretes, e não p

c:\Users\Tales\Python\AgenteTuristico\.venv\lib\site-packages\langchain_google_genai\chat_models.py:310: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


```tool_code
Thought: Do I need to use a tool? No
Final Answer: Ótimo!  Com base nas informações que você me deu, aqui está um roteiro agitado para sua viagem de 4 dias a Natal, focado em praias e restaurantes regionais, com um orçamento de R$600,00 (excluindo hospedagem):

**Dia 1 (Quarta-feira, 01/01/2024): Chegada e Ponta Negra**

* **Manhã:** Chegada em Natal, acomodação (não incluída no orçamento) e partida para Ponta Negra.
* **Tarde:** Relaxe na Praia de Ponta Negra, aproveite o sol e o mar.  Considere um mergulho ou um passeio de jangada (verifique preços para adequar ao orçamento).
* **Noite:** Jantar no Restaurante Farofa d'Água em Ponta Negra, apreciando a vista do Morro do Careca.  (Verifique o cardápio e preços para garantir que se encaixa no seu orçamento).


**Dia 2 (Quinta-feira, 02/01/2024): Passeio Panorâmico e Porto Mirim**

* **Manhã:** Passeio Panorâmico de Natal, incluindo visita à Praia de Porto Mirim.  Pesquise opções de passeios que se encaixem no seu orçamento

In [ ]:
print(response["chat_history"])